In [191]:
print('Lets build the model :)')
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
#In this modeling approach we are taking 3 previous and trying to predict the 4th word in the sequance

Lets build the model :)


In [192]:
# reading the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [193]:
len(words)

32033

In [194]:
# building the mappings of characters to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [195]:
#building the dataset
block_size = 3 #context length how many characters do we take to predict the next one
X,Y = [],[]
for w in words[:5]:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

        

In [196]:
g = torch.Generator().manual_seed(219384384)
#creating a lookup table for the character embeddings
C = torch.randn((27,2), generator = g)
# Constructiong the hidden layer
#The number of inputs to this layer is going to be 3 * 2  because we hae to dimensional embeddings and we have 3 of them
# and its up to us to decide how many neurons we want inside the layer here we are going with 100 of them
W1 = torch.randn(6,100, generator = g )
b1 = torch.randn(100, generator = g) 
W2 = torch.randn(100, 27, generator = g) #our second layer will take 100 inputs and 
b2 = torch.randn(27, generator = g)
parameters = [C,W1, b1, W2, b2]

In [197]:
sum(p.nelement() for p in parameters) #tells us number of parameters in total

3481

In [198]:
emb = C[X] #the shape is (32,3,2)
h = torch.tanh(emb.view(-1,6) @ W1 + b1) #we are using tanh activation fn so the numbers in h will be -1 and 1 
# We pass -1 to emb.view(), becuase we want pytorch to guess what what will be the right number for rows given we have already told it the number of columns are 6
# here you can also use emb.reshape(32,6) the difference is emb.view() will not use extra space it ensure that the emb tensor and and the new tensor that we create will use the same data so no memory wastag
logits = h @ W2 + b2 #the shape is 32,27
counts = logits.exp() 
prob = counts/ counts.sum(dim = 1, keepdim= True)
# implementing the loss function
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(18.5619)